In [3]:
import pandas as pd

In [4]:
!pwd

/Users/Rrando/Documents/GitHub/smart-tab-grouping/notebooks


In [5]:
fine_tune_raw = pd.read_csv("../data/extract/fine_tuning_data__with_none__common_crawl_2025-02-23_08-18.csv", keep_default_na=False)

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
import numpy as np

In [8]:
hint_embedder = pipeline("feature-extraction", model="sentence-transformers/all-MiniLM-L6-v2", device=-1)


Device set to use cpu


In [9]:
def get_embeddings_for_hint_matching(input_df):
    embed_input = pd.Series(input_df["input_titles"] + " " + input_df["input_keywords"]).to_list()
    return np.array([np.mean(hint_embedder(sentence)[0], axis=0) for sentence in embed_input])


In [10]:
per_row_embeddings = get_embeddings_for_hint_matching(fine_tune_raw)

In [27]:
FIXED_TOPICS = {"Adult Content", "None"}

In [32]:
def refine_table_class(row, index):
    
    initial = row.output
    if initial in FIXED_TOPICS:
        return initial
    words = initial.split()
    initial_embedding = hint_embedder(initial)[0][0]
    if len(words) == 1:
        return initial
    alt_embeddings = [initial_embedding]
    alt_phrases = [initial]
    for missing_word_index in range(len(words)):
        # remove 1 word
        shortened_phrase = " ".join(words[:missing_word_index] + words[missing_word_index+1:])
        embed = hint_embedder(shortened_phrase)[0][0]
        alt_phrases.append(shortened_phrase)
        alt_embeddings.append(embed)
    if len(words) > 2:
        # remove 2 words
        for missing_word_index in range(len(words) - 1):
            shortened_phrase = " ".join(words[:missing_word_index] + words[missing_word_index + 2:])
            embed = hint_embedder(shortened_phrase)[0][0]
            alt_phrases.append(shortened_phrase)
            alt_embeddings.append(embed)
    document_embedding = per_row_embeddings[index].reshape(1,-1)
    similarity = cosine_similarity(document_embedding, np.array(alt_embeddings)).squeeze()
    similarity[0] -= 0.1 # was 0.1
    closest_indices = np.argsort(-similarity)
    best_phrases = [alt_phrases[i] for i in closest_indices.tolist()]
    return best_phrases[0]


In [33]:
for index, row in fine_tune_raw.iterrows():
    print(refine_row(row, index))
    if index > 500:
        break

None
None
None
Science
Watches
Watches
User Management
None
Anxiety Support
Sports
Botanical
Account
Battery
Travel
Blog
Summer Travel
Blogging
Data
Sales
Musical Accessories
Reading
Food Equipment
Book
None
Microsoft News
Music Cards
Poetry
Astrology
Music
Short Film
Blogging
Photo Gallery
Book
Treats
Business
Course Reviews
Terms
None
Poultry
Pajamas
Dating Review
Wedding Photography
Gaming
Records
None
Infrastructure Growth
Belts Supply
Business
Janitorial
Player
None
Photography
Homestuck
Discussions
Settings
Political
Health
Nature
Promotions
News
None
Art
Attractions
Remote Portfolio
Leadership
Rankings
Decor
Puzzles
Blog Tour
Job Listings
Blog
Heritage
Heat Pumps
Cheese
Nutrition
Information
Packrat Guide
None
Automotive
Art
Locksmith
Local
Development
Replacement Child
Uncategorized
Search
Airline Branding
Flower
Gaming
Estate Listings
Product Information
Gaming
None
Indie Hackers
Home
Comics
Medical Testing
Gear
Spirituality
Data
Decor
Adult Content
Environmental
Demolition
De

In [ ]:
refine_row(fine_tune_raw.iloc[0])

In [ ]:
def remove_and(a: str):
    words = a.split(" ")
    if len(words) < 2:
        return a
    for stopword in ["and", "&"]:
        if stopword in words:
            and_pos = words.index(stopword)
            if and_pos > 0:
                return " ".join(words[0:and_pos])
    return " ".join(words)


In [ ]:
remove_and("Word & Dog")

In [ ]:
def remove_or(a: str):
    words = a.split(" ")
    if len(words) < 3:
        return a
    if words[-2] == 'or':
        words = words[:-2]
    return " ".join(words)


In [ ]:
def remove_news(a: str):
    words = a.split(" ")
    if len(words) == 1:
        return a
    if words[-1] == 'News':
        words = words[:-1]
    return " ".join(words)


In [ ]:
remove_news("Tech News")

In [ ]:
def remove_year(a: str):
    words = list(filter(lambda word: word not in ["2023", "2024"], a.split(" ")))
    return " ".join(words)
